In [12]:
from pyspark.sql import SparkSession


In [18]:
spark = (
    SparkSession
    .builder
    .appName("Unique Data and some pyspark sql window functions")
    .master("local[*]")
    .getOrCreate()
    )

In [19]:
spark

In [20]:
emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [23]:
emp = spark.createDataFrame(data = emp_data, schema = emp_schema)

In [25]:
emp_dist = emp.distinct()

In [26]:
emp_dist.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [30]:
from pyspark.sql.functions import col

In [33]:
emp_dept_id = emp.select("department_id").distinct()

In [34]:
emp_dept_id.show()

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          103|
|          104|
|          105|
|          106|
|          107|
+-------------+



In [37]:
from pyspark.sql.window import Window
from pyspark.sql.functions import max, col, desc

window_spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())
max_fun = max(col("salary")).over(window_spec)

emp1 = emp.withColumn("max_salary", max_fun)

In [39]:
emp1.show(5)

+-----------+-------------+-------------+---+------+------+----------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+-------------+---+------+------+----------+----------+
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|     70000|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|     70000|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|     70000|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|     55000|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|     55000|
+-----------+-------------+-------------+---+------+------+----------+----------+
only showing top 5 rows



In [40]:
from pyspark.sql.functions import row_number, desc, col

In [46]:
window_spec = Window.partitionBy(col("department_id")).orderBy(col("salary").desc())

rn = row_number().over(window_spec)

emp2 = emp.withColumn("row_number", rn).where("row_number = 2")

In [47]:
emp2.show()

+-----------+-------------+-----------+---+------+------+----------+----------+
|employee_id|department_id|       name|age|gender|salary| hire_date|row_number|
+-----------+-------------+-----------+---+------+------+----------+----------+
|        001|          101|   John Doe| 30|  Male| 50000|2015-01-01|         2|
|        020|          102|  Grace Kim| 32|Female| 53000|2018-11-01|         2|
|        005|          103|  Jack Chan| 40|  Male| 60000|2013-04-01|         2|
|        018|          104|  Nancy Liu| 29|      | 50000|2017-06-01|         2|
|        012|          105| Susan Chen| 31|Female| 54000|2017-02-15|         2|
|        015|          106|Michael Lee| 37|  Male| 63000|2014-09-30|         2|
|        014|          107|  Emily Lee| 26|Female| 46000|2019-01-01|         2|
+-----------+-------------+-----------+---+------+------+----------+----------+



In [48]:
# usign expr

from pyspark.sql.functions import expr

emp3 = emp.withColumn("rn", expr("row_number() over(partition by department_id order by salary desc)"))

In [49]:
emp3.show()

+-----------+-------------+-------------+---+------+------+----------+---+
|employee_id|department_id|         name|age|gender|salary| hire_date| rn|
+-----------+-------------+-------------+---+------+------+----------+---+
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|  1|
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|  2|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|  3|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|  1|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|  2|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|  3|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|  4|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|  1|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|  2|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|  3|
|        006|          10